In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_163624\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_00720s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-671.267311,-90.259375,1,3,2091.391496,3823.292859,0.345128,0.321786,1,-671.267311,"-1725,-2",-27.173147,-895.245495,-5.995013,-74.191610,0.762949
1,2,1,1,NaN,-425.041608,-77.950447,0,2,2090.529471,3821.000000,0.499131,0.000061,1,-425.041608,"-1725,-1",-26.737013,-894.089744,-5.826496,-73.992142,0.508633
2,3,1,1,NaN,-687.554293,-83.046325,0,3,2086.313863,3820.966777,0.328141,0.574388,1,-687.554293,-1725,-24.611071,-894.075595,-5.361884,-73.992423,0.762949
3,4,1,1,NaN,-395.425159,-71.558915,0,2,1926.524284,3819.000000,0.499410,0.000000,0,-395.425159,None,55.971649,-893.182484,12.109001,-73.776935,0.508633
4,5,1,1,NaN,-944.981250,-93.609713,2,4,1929.280999,3813.995890,0.259511,0.386099,0,-944.981250,None,54.584536,-890.657219,11.509380,-73.355528,1.017266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364691,701951,97,8,NaN,177.679721,32.113240,0,4,164.758332,409.804854,0.247160,0.477270,1,NaN,630456,-370.563185,623.588570,-28.929372,36.776168,65.265039
1364692,701952,97,8,NaN,111.662679,28.069888,0,2,345.000000,441.518841,0.000000,0.499645,0,NaN,None,357.494017,751.692114,34.137207,48.195936,32.632520
1364693,701953,97,8,NaN,232.943305,33.880059,1,3,371.120203,443.284456,0.580803,0.319015,0,NaN,None,463.002370,758.824031,48.609404,49.747992,48.948779
1364694,701954,97,8,NaN,96.178861,28.389323,0,2,272.525335,450.000000,0.499358,0.000005,0,NaN,None,64.744276,785.950392,6.152600,50.810670,32.632520


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 533792/533792 [2:01:22<00:00, 73.29it/s]  


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_00720s.pnds')